In [ ]:
import pandas as pd
import numpy as np

pathRoot2 = "** PATH TO CSV FILES USED **"
secondBatchList = ["1", "2", "3", "4", "5", "6"]
csvEnd = ".csv"

def format(x):
    return '${:0,.0f}'.format(x)

for file in secondBatchList:
    df = pd.read_csv(pathRoot2 + file + csvEnd)
    df.drop(df.columns.difference(['LoanAmount', 'BusinessName', 'City', 'State']), 1, inplace=True)
    df.rename(columns = {'BusinessName':'Recipient'}, inplace = True)
    df.rename(columns = {'LoanAmount':'Amount'}, inplace = True)
    df['Source'] = "PPP"
    df = df[['Recipient', 'City', 'State', 'Source', 'Amount']]
    df['Amount'] = df['Amount'].apply(format)
    df['Batch'] = str(2)
    df['File'] = file
    df['Row'] = df.index + 2
    df.to_csv(pathRoot2 + file + "tempCSV.csv", index=False)

In [ ]:
import pandas as pd

pathRoot1 = "** PATH TO CSV FILES USED **"
firstBatchList = ["1", "2", "3", "4", "5", "6", "7", "8"]
xlEnd = ".xlsx"

def amountMessage(x):
    if x == "PPP":
        return "$5-$10 million"
    else:
        return "Not Disclosed"

for file in firstBatchList:
    df = pd.read_excel(pathRoot1 + file + xlEnd)
    df['Amount'] = df['Source'].apply(amountMessage)
    df['Batch'] = str(1)
    df['File'] = file
    df['Row'] = df.index + 2
    df.to_csv(pathRoot1 + file + "tempCSV.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd

pathRoot1 = "** PATH TO CSV FILE USED **"
firstBatchList = ["1", "2", "3", "4", "5", "6", "7", "8"]
xlEnd = ".xlsx"


def format(x):
    return '${:0,.0f}'.format(x)


df = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])
for firstBatchFile in firstBatchList:
    tempDF = pd.read_csv(pathRoot1 + firstBatchFile + "tempCSV.csv")
    df = df.append(tempDF)
splitDF_PSP = df[df.Source == "PSP"]
splitDF_PSP = splitDF_PSP.reset_index(drop=True)
df = df[df.Source != "PSP"]


amountDF = pd.read_excel(pathRoot1 + "** REDACTED **" + xlEnd)
c = 1
for col in amountDF.columns:
    amountDF.rename(columns = {col : str(c)}, inplace = True)
    c += 1
amountDF.drop(amountDF.columns.difference(['1', '3', '5']), 1, inplace=True)
amountDF.rename(columns = {'1':'Recipient'}, inplace = True)
amountDF.rename(columns = {'3':'State'}, inplace = True)
amountDF.rename(columns = {'5':'Amount'}, inplace = True)
amountDF['Amount'] = amountDF['Amount'].apply(format)


print(splitDF_PSP)
print(amountDF)


splitDF_PSP['Amount'] = amountDF['Amount']
df = df.append(splitDF_PSP)
print(splitDF_PSP)
print(df)

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from string import punctuation


print("Compiling...")

startTime = datetime.now()

specialCharacters = set(punctuation)
numberCharacters = str(set(range(10)))

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

pathRoot1 = "** PATH TO CSV FILES USED **"
pathRoot2 = "** PATH TO MORE CSV FILES USED **"
pathRootFinal = "** PATH TO OUTPUTS **"
outputName = "output__"
firstBatchList = ["1", "2", "3", "4", "5", "6", "7", "8"]
secondBatchList = ["1", "2", "3", "4", "5", "6"]
xlEnd = ".xlsx"
csvEnd = ".csv"


def cleanser(recipient):
    for char in specialCharacters:
        recipient = recipient.replace(char, "")
    for num in numberCharacters:
        if len(recipient) > 0:
            if recipient[0] == num:
                recipient = recipient[1:]
    recipient = ' '.join(recipient.split())
    return recipient.upper()


def comparator(name1, name2):
    components1 = name1.split(" ")
    components2 = name2.split(" ")
    if len(components1) <= 1:
        if len(components2) >= 1:
            return False
    elif len(components2) <= 1:
        if len(components1) >= 1:
            return False
    else:
        if len(components1) >= len(components2):
            iteration = components2
            relation = components1
        else:
            iteration = components1
            relation = components2
        matchCount = 0
        for a in range(len(iteration)):
            if iteration[a] == relation[a]:
                matchCount += 1
            elif iteration[a][:-1] == relation[a]:
                matchCount += 1
            elif iteration[a] == relation[a][:-1]:
                matchCount += 1
        return matchCount / len(iteration) >= 0.80
    
def format(x):
    return '${:0,.0f}'.format(x)


df = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])

for firstBatchFile in firstBatchList:
    tempDF = pd.read_csv(pathRoot1 + firstBatchFile + "tempCSV.csv")
    df = df.append(tempDF)
    
splitDF_PSP = df[df.Source == "PSP"]
splitDF_PSP = splitDF_PSP.reset_index(drop=True)
df = df[df.Source != "PSP"]
df = df.reset_index(drop=True)

amountDF = pd.read_excel(pathRoot1 + "** REDACTED **" + xlEnd)
c = 1
for col in amountDF.columns:
    amountDF.rename(columns = {col : str(c)}, inplace = True)
    c += 1
amountDF.drop(amountDF.columns.difference(['1', '3', '5']), 1, inplace=True)
amountDF.rename(columns = {'1':'Recipient'}, inplace = True)
amountDF.rename(columns = {'3':'State'}, inplace = True)
amountDF.rename(columns = {'5':'Amount'}, inplace = True)
amountDF['Amount'] = amountDF['Amount'].apply(format)

splitDF_PSP['Amount'] = amountDF['Amount']
df = df.append(splitDF_PSP)

newDF = pd.read_excel(pathRoot1 + "** REDACTED **" + xlEnd)
c = 1
for col in newDF.columns:
    newDF.rename(columns = {col : str(c)}, inplace = True)
    c += 1
newDF.drop(newDF.columns.difference(['1', '2', '3', '5']), 1, inplace=True)
newDF.rename(columns = {'1':'Recipient'}, inplace = True)
newDF.rename(columns = {'2':'City'}, inplace = True)
newDF.rename(columns = {'3':'State'}, inplace = True)
newDF.rename(columns = {'5':'Amount'}, inplace = True)
newDF['Source'] = "PSP"
newDF = newDF[['Recipient', 'City', 'State', 'Source', 'Amount']]
newDF['Amount'] = newDF['Amount'].apply(format)
newDF['Batch'] = 3
newDF['File'] = str(1)
newDF['Row'] = newDF.index + 2

df = df.append(newDF)

for secondBatchFile in secondBatchList:
    tempDF = pd.read_csv(pathRoot2 + secondBatchFile + "tempCSV.csv")
    df = df.append(tempDF)

df = df[df.Recipient != "#NAME?"]
df = df.reset_index(drop=True)



outputFiles = []
outputFiles1 = []
outputFiles2 = []
outputFiles3 = []
for state in states:
    incStartTime = datetime.now()
    result = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row', 'Recipient (Cleansed)'])
    result1 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row', 'Recipient (Cleansed)'])
    result2 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row', 'Recipient (Cleansed)'])
    result3 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row', 'Recipient (Cleansed)'])
    print(state + " compiling...")
    coreDF = df[df.State == state]
    dfRef = pd.DataFrame(coreDF[coreDF.Source != "PSP"])
    dfKey = pd.DataFrame(coreDF[coreDF.Source == "PSP"])
    if len(dfRef.Recipient) > 0 and len(dfKey.Recipient) > 0:
        dfRef['Recipient (Cleansed)'] = dfRef.apply(lambda row: cleanser(str(row.Recipient)), axis=1)
        dfKey['Recipient (Cleansed)'] = dfKey.apply(lambda row: cleanser(str(row.Recipient)), axis=1)
        dfKeyArray = dfKey["Recipient (Cleansed)"].to_numpy()
        dfRefArray = dfRef["Recipient (Cleansed)"].to_numpy()
        for i in range(len(dfKeyArray)):
            for j in range(len(dfRefArray)):
                if i == j:
                    continue
                else:
                    if comparator(dfKeyArray[i], dfRefArray[j]):
                        result = result.append(dfKey.iloc[i])
                        result = result.append(dfRef.iloc[j])
                        if dfRef.iloc[j,5] == 1:
                            result1 = result1.append(dfKey.iloc[i])
                            result1 = result1.append(dfRef.iloc[j])
                        elif dfRef.iloc[j,5] == 2:
                            result2 = result2.append(dfKey.iloc[i])
                            result2 = result2.append(dfRef.iloc[j])
                        if dfKey.iloc[i,5] == 3:
                            result3 = result3.append(dfKey.iloc[i])
                            result3 = result3.append(dfRef.iloc[j])
    result = result.drop(['Recipient (Cleansed)'], axis=1)
    result = result.drop_duplicates()
    result1 = result1.drop(['Recipient (Cleansed)'], axis=1)
    result1 = result1.drop_duplicates()
    result2 = result2.drop(['Recipient (Cleansed)'], axis=1)
    result2 = result2.drop_duplicates()
    result3 = result3.drop(['Recipient (Cleansed)'], axis=1)
    result3 = result3.drop_duplicates()
    writePath = pathRootFinal + outputName + state + csvEnd
    writePath1 = pathRootFinal + outputName + state + "_B1" + csvEnd
    writePath2 = pathRootFinal + outputName + state + "_B2" + csvEnd
    writePath3 = pathRootFinal + outputName + state + "_B3" + csvEnd
    outputFiles.append(writePath)
    outputFiles1.append(writePath1)
    outputFiles2.append(writePath2)
    outputFiles3.append(writePath3)
    result.to_csv(writePath, index=False)
    result1.to_csv(writePath1, index=False)
    result2.to_csv(writePath2, index=False)
    result3.to_csv(writePath3, index=False)
    print("  " + state + " compiled and written to Excel : " + outputName + state + csvEnd)
    incEndTime = datetime.now()
    incElapsed = incEndTime - incStartTime
    print("    Elapsed Time for " + state + "  :  " + str(incElapsed))
    print(" ")

    
print("Completed compiling.")
print(" ")
print("Merging CSV Files into one Excel Workbook...")


odf = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])
for outputFile in outputFiles:
    oidf = pd.read_csv(outputFile)
    odf = odf.append(oidf)
odf.to_excel(pathRootFinal + "DoubleDippers_All.xlsx", index=False)
odf.to_csv(pathRootFinal + "DoubleDippers_All.csv", index=False)

odf1 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])
for outputFile1 in outputFiles1:
    oidf1 = pd.read_csv(outputFile1)
    odf1 = odf1.append(oidf1)
odf1.to_excel(pathRootFinal + "DoubleDippers_Batch_1.xlsx", index=False)
odf1.to_csv(pathRootFinal + "DoubleDippers_Batch_1.csv", index=False)

odf2 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])
for outputFile2 in outputFiles2:
    oidf2 = pd.read_csv(outputFile2)
    odf2 = odf2.append(oidf2)
odf2.to_excel(pathRootFinal + "DoubleDippers_Batch_2.xlsx", index=False)
odf2.to_csv(pathRootFinal + "DoubleDippers_Batch_2.csv", index=False)

odf3 = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])
for outputFile3 in outputFiles3:
    oidf3 = pd.read_csv(outputFile3)
    odf3 = odf3.append(oidf3)
odf3.to_excel(pathRootFinal + "DoubleDippers_Batch_3.xlsx", index=False)
odf3.to_csv(pathRootFinal + "DoubleDippers_Batch_3.csv", index=False)


print("Published outputs available in .xlsx format")
print(" ")
print("Backup outputs available in .csv format")    

endTime = datetime.now()
elapsed = endTime - startTime

print(" ")
print("Elapsed Time  :  " + str(elapsed))

Compiling...


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AL compiling...
  AL compiled and written to Excel : output__AL.csv
    Elapsed Time for AL  :  0:00:09.713399
 
AK compiling...
  AK compiled and written to Excel : output__AK.csv
    Elapsed Time for AK  :  0:00:09.500603
 
AZ compiling...
  AZ compiled and written to Excel : output__AZ.csv
    Elapsed Time for AZ  :  0:00:16.646829
 
AR compiling...
  AR compiled and written to Excel : output__AR.csv
    Elapsed Time for AR  :  0:00:05.259742
 
CA compiling...
  CA compiled and written to Excel : output__CA.csv
    Elapsed Time for CA  :  0:07:19.212256
 
CO compiling...
  CO compiled and written to Excel : output__CO.csv
    Elapsed Time for CO  :  0:00:20.219627
 
CT compiling...
  CT compiled and written to Excel : output__CT.csv
    Elapsed Time for CT  :  0:00:08.445229
 
DC compiling...
  DC compiled and written to Excel : output__DC.csv
    Elapsed Time for DC  :  0:00:00.989501
 
DE compiling...
  DE compiled and written to Excel : output__DE.csv
    Elapsed Time for DE  :  

In [ ]:
## CHECK DOUBLE_DIPPERS_ALL AGAINST THE 8 ORIGINAL FILES


import numpy as np
import pandas as pd
from datetime import datetime
from string import punctuation


print("Compiling...")

startTime = datetime.now()

specialCharacters = set(punctuation)
numberCharacters = str(set(range(10)))

#states = ["CA"]

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

pathRoot1 = "** PATH TO CSV FILES USED **"
pathRoot2 = "** PATH TO MORE CSV FILES USED **"
pathRootFinal = "** PATH TO OUTPUTS **"
outputName = "output__"
firstBatchList = ["1", "2", "3", "4", "5", "6", "7", "8"]
secondBatchList = ["1", "2", "3", "4", "5", "6"]
xlEnd = ".xlsx"
csvEnd = ".csv"



def format(x):
    return '${:0,.0f}'.format(x)


df = pd.DataFrame(columns=['Recipient', 'City', 'State', 'Source', 'Amount', 'Batch', 'File', 'Row'])

for firstBatchFile in firstBatchList:
    tempDF = pd.read_csv(pathRoot1 + firstBatchFile + "tempCSV.csv")
    df = df.append(tempDF)
    
splitDF_PSP = df[df.Source == "PSP"]
splitDF_PSP = splitDF_PSP.reset_index(drop=True)
df = df[df.Source != "PSP"]



newDF = pd.read_excel(pathRoot1 + "** REDACTED **" + xlEnd)
c = 1
for col in newDF.columns:
    newDF.rename(columns = {col : str(c)}, inplace = True)
    c += 1
newDF.drop(newDF.columns.difference(['1', '2', '3', '5']), 1, inplace=True)
newDF.rename(columns = {'1':'Recipient'}, inplace = True)
newDF.rename(columns = {'2':'City'}, inplace = True)
newDF.rename(columns = {'3':'State'}, inplace = True)
newDF.rename(columns = {'5':'Amount'}, inplace = True)
newDF['Source'] = "PSP"
newDF = newDF[['Recipient', 'City', 'State', 'Source', 'Amount']]
newDF['Amount'] = newDF['Amount'].apply(format)
newDF['Batch'] = 3
newDF['File'] = str(1)
newDF['Row'] = newDF.index + 2



checkDF = newDF[newDF['Recipient'].isin(splitDF_PSP['Recipient'])]
checkDF1 = newDF[newDF['Recipient'].isin(df['Recipient'])]
checkDF2 = df[df['Recipient'].isin(newDF['Recipient'])]


print(len(df))
print(len(newDF))
print(len(checkDF))
print(len(checkDF1))
print(len(checkDF2))

df.to_csv(pathRootFinal + "test-OTHERs.csv")
splitDF_PSP.to_csv(pathRootFinal + "test-PSPs.csv")
newDF.to_csv(pathRootFinal + "test-cargo-contractors.csv")
checkDF.to_csv(pathRootFinal + "test-checkDF.csv")
checkDF1.to_csv(pathRootFinal + "test-checkDF1.csv")
checkDF2.to_csv(pathRootFinal + "test-checkDF2.csv")